作業の流れ  
①slowの場合　実際の再生速度を確認  
  25fpsのslow→2倍速で通常なら50fpsに変換

②50fps→30fpsに画像を間引く  
③preGT付与
④GT依頼  
⑤2倍にする  


In [2]:
import cv2
import os
import sys
import time


# fps変更して保存

In [18]:
# 動画を読み込み、FPSを変更して別名で保存する関数
def m_speed_change(path_in, path_out, fps_new, color_flag):
    # 動画読み込みの設定
    movie = cv2.VideoCapture(path_in)
 
    # 動画ファイル保存用の設定
    fps = int(round(movie.get(cv2.CAP_PROP_FPS)))
    #print(fps)# 元動画のFPSを取得
    #fps_new = int(fps * scale_factor)                                       # 動画保存時のFPSはスケールファクターをかける
    w = int(movie.get(cv2.CAP_PROP_FRAME_WIDTH))                            # 動画の横幅を取得
    h = int(movie.get(cv2.CAP_PROP_FRAME_HEIGHT))                           # 動画の縦幅を取得
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')                     # 動画保存時のfourcc設定（mp4用）
    video = cv2.VideoWriter(path_out, fourcc, fps_new, (w, h), color_flag)  # 動画の仕様（ファイル名、fourcc, FPS, サイズ）
 
    # ファイルからフレームを1枚ずつ取得して動画処理後に保存する
    while True:
        ret, frame = movie.read()        # フレームを取得
        video.write(frame)               # 動画を保存する
        # フレームが取得できない場合はループを抜ける
        if not ret:
            break
    # 撮影用オブジェクトとウィンドウの解放
    movie.release()
    return
 
#path_in = 'Mountain - 34608.mp4'          # 元動画のパス
#path_in = 'Elster - 35908.mp4' 
#保存ファイル用
#name = os.path.splitext(path_in)
#fps_new=30# FPSにかけるスケールファクター
#path_out = name[0]+"_{}fps".format(fps_new)+name[1] # 保存する動画のパス
#scale_factor = 2
#color_flag = True               # カラー動画はTrue, グレースケール動画はFalse
 
# 動画の再生速度を変更する関数を実行
#m_speed_change(path_in, path_out, fps_new, color_flag)

In [44]:
path_in = 'Elster - 35908.mp4'#元ファイルパス
#保存ファイル生成用
name = os.path.splitext(path_in)
movie = cv2.VideoCapture(path_in)
fps = (movie.get(cv2.CAP_PROP_FPS))

#下記整数倍のfps動画生成
fps_times = [2,3,4,5]#fpsのx倍のリスト
#fps_times = [1,1.5,2,2.5,3,3.5,4,4.5,5]#fpsのx倍のリスト
fpss = list(map(lambda x:x*fps,fps_times))
color_flag = True  

for i in fpss:
    path_out = name[0]+"_{0:.1f}fps".format(i)+name[1] # 保存する動画のパス
    print("{}fps".format(path_out))
    m_speed_change(path_in, path_out, i, color_flag)

59.9fps
89.9fps
119.9fps
149.9fps


# 30fps相当の画像保存

#120fps →30fps  
target_fps = 30  
fps = cap.get(cv2.CAP_PROP_FPS)  
step_times = fps/target_fps  #間隔が何倍か  
step_sec = fps_inv*_times　#間隔（秒）  

In [72]:
def save_frame_range_sec_target_fps(video_path, target_fps, start_sec, stop_sec, 
                         dir_path, basename, ext='jpg'):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    fps = cap.get(cv2.CAP_PROP_FPS)
    fps_inv = 1 / fps
    
    #指定fpsの間隔
    step_times = fps/target_fps  #間隔が何倍か
    step_sec = fps_inv*step_times #間隔（秒）
    print("1/{}倍のfpsで画像生成".format(step_times))
    print("{:.2f}秒間隔".format(step_sec))
    
    count = 0

    sec = start_sec
    while sec < stop_sec:
        n = round(fps * sec)#開始秒
        cap.set(cv2.CAP_PROP_POS_FRAMES, n)#開始位置へ移動
        ret, frame = cap.read()
        if ret:
            cv2.imwrite(
                '{}_{}_{:.2f}_{}.{}'.format(
                    base_path, str(n).zfill(digit), n * fps_inv, str(count).zfill(digit), ext
                ),
                frame
            )
        else:
            print("Finished")
            return
        count += 1
        sec += step_sec

In [73]:
path_in = 'Elster - 35908_119.9fps.mp4'
directory = "C:\\Users\\Ten\\python\\work\\sample2\\"

save_frame_range_sec_target_fps(path_in,30, 0, 100,
                     directory, 'sample_video_img')

1/3.996倍のfpsで画像生成
0.03秒間隔
Finished


# 以下、参考

# 再生

In [42]:
#file_path = 'Mountain - 34608.mp4'
file_path = "Elster - 35908.mp4"
cap = cv2.VideoCapture(file_path)
#fps = 30
fps = cap.get(cv2.CAP_PROP_FPS)
if cap.isOpened()== False:
    print("Error!")

while cap.isOpened():

    ret, frame = cap.read()

    if ret == True:

        time.sleep(1/fps)
        cv2.imshow('frame',frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):

            break

    else:
        break

cap.release()
cv2.destroyAllWindows()

In [48]:
import numpy as np
 
def m_slice(path, dir, step, extension):
    movie = cv2.VideoCapture(path)                  # 動画の読み込み
    Fs = int(movie.get(cv2.CAP_PROP_FRAME_COUNT))   # 動画の全フレーム数を計算
    path_head = dir + '\\'                       # 静止画のファイル名のヘッダー
    ext_index = np.arange(0, Fs, step)              # 静止画を抽出する間隔
 
    for i in range(Fs - 1):                         # フレームサイズ分のループを回す
        flag, frame = movie.read()                  # 動画から1フレーム読み込む
        check = i == ext_index                      # 現在のフレーム番号iが、抽出する指標番号と一致するかチェックする
        
        # frameを取得できた(flag=True)時だけ処理を行う
        if flag == True:
            # もしi番目のフレームが静止画を抽出するものであれば、ファイル名を付けて保存する
            if True in check:
                # ファイル名は後でフォルダ内で名前でソートした時に連番になるようにする
                path_out = path_head + str(i).zfill(5) + extension
                cv2.imwrite(path_out, frame)
                print(path_out)
            # i番目のフレームが静止画を抽出しないものであれば、何も処理をしない
            else:
                pass
        else:
            pass
    return
 
# 関数実行：引数=（ファイル名のパス、保存先のフォルダパス、ステップ数、静止画拡張子）
path_in = 'Elster - 35908_119.9fps.mp4'
directory = "C:\\Users\\Ten\\python\\work\\fps変換\\out"
m_slice(path_in, directory, 50, '.jpg')

C:\Users\Ten\python\work\fps変換\out\00000.jpg
C:\Users\Ten\python\work\fps変換\out\00050.jpg
C:\Users\Ten\python\work\fps変換\out\00100.jpg
C:\Users\Ten\python\work\fps変換\out\00150.jpg
C:\Users\Ten\python\work\fps変換\out\00200.jpg
C:\Users\Ten\python\work\fps変換\out\00250.jpg
C:\Users\Ten\python\work\fps変換\out\00300.jpg
C:\Users\Ten\python\work\fps変換\out\00350.jpg
C:\Users\Ten\python\work\fps変換\out\00400.jpg
C:\Users\Ten\python\work\fps変換\out\00450.jpg
C:\Users\Ten\python\work\fps変換\out\00500.jpg
C:\Users\Ten\python\work\fps変換\out\00550.jpg
C:\Users\Ten\python\work\fps変換\out\00600.jpg
C:\Users\Ten\python\work\fps変換\out\00650.jpg
C:\Users\Ten\python\work\fps変換\out\00700.jpg
C:\Users\Ten\python\work\fps変換\out\00750.jpg
C:\Users\Ten\python\work\fps変換\out\00800.jpg
C:\Users\Ten\python\work\fps変換\out\00850.jpg
C:\Users\Ten\python\work\fps変換\out\00900.jpg
C:\Users\Ten\python\work\fps変換\out\00950.jpg
C:\Users\Ten\python\work\fps変換\out\01000.jpg
C:\Users\Ten\python\work\fps変換\out\01050.jpg
C:\Users\T

# (うまく動作しない）再生fps変更

In [41]:

#file_path = 'Mountain - 34608.mp4'
file_path = "Elster - 35908.mp4"
cap = cv2.VideoCapture(file_path)
fps_ref = cap.get(cv2.CAP_PROP_FPS)
print("元のfps{}".format(fps_ref))# 元動画のFPSを取得
fps = fps_ref*6
print("再生{}fps".format(fps))
#fps = cap.get(cv2.CAP_PROP_FPS)
if cap.isOpened()== False:
    print("Error!")

while cap.isOpened():

    ret, frame = cap.read()

    if ret == True:

        time.sleep(1/fps)
        cv2.imshow('frame',frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):

            break

    else:
        break

cap.release()
cv2.destroyAllWindows()

元のfps29.97002997002997
再生179.82017982017982fps


# （うまく動作しない）動画再生時のFPSを測定、描画して表示

In [52]:
camera_id = 0
delay = 1
window_name = 'frame'

#fps = 120
#fps = cap.get(cv2.CAP_PROP_FPS)

cap = cv2.VideoCapture(file_path)

if not cap.isOpened():
    sys.exit()

tm = cv2.TickMeter()
tm.start()

count = 0
max_count = 20
fps = 0

while cap.isOpened():
    ret, frame = cap.read()

    if count == max_count:
        tm.stop()
        fps = max_count / tm.getTimeSec()
        tm.reset()
        tm.start()
        count = 0

    cv2.putText(frame, 'FPS: {:.2f}'.format(fps),
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), thickness=2)
    cv2.imshow(window_name, frame)
    count += 1

    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break
    

cv2.destroyWindow(window_name)
cap.release()
cv2.destroyAllWindows()

error: OpenCV(3.4.2) C:\Miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\highgui\src\window.cpp:356: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [2]:
import cv2 
cv2.__version__

'3.4.2'

In [49]:
# 動画を読み込み、FPSを変更して別名で保存する関数
def m_speed_change(path_in, path_out, scale_factor, color_flag):
    # 動画読み込みの設定
    movie = cv2.VideoCapture(path_in)
 
    # 動画ファイル保存用の設定
    fps = int(movie.get(cv2.CAP_PROP_FPS))                                  # 元動画のFPSを取得
    fps_new = fps * scale_factor                                       # 動画保存時のFPSはスケールファクターをかける
    w = int(movie.get(cv2.CAP_PROP_FRAME_WIDTH))                            # 動画の横幅を取得
    h = int(movie.get(cv2.CAP_PROP_FRAME_HEIGHT))                           # 動画の縦幅を取得
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')                     # 動画保存時のfourcc設定（mp4用）
    video = cv2.VideoWriter(path_out, fourcc, fps_new, (w, h), color_flag)  # 動画の仕様（ファイル名、fourcc, FPS, サイズ）
 
    # ファイルからフレームを1枚ずつ取得して動画処理後に保存する
    while True:
        ret, frame = movie.read()        # フレームを取得
        video.write(frame)               # 動画を保存する
        # フレームが取得できない場合はループを抜ける
        if not ret:
            break
    # 撮影用オブジェクトとウィンドウの解放
    movie.release()
    return
 
path_in = 'Mountain - 34608.mp4'          # 元動画のパス
path_out = 'video_out.mp4'      # 保存する動画のパス
scale_factor = 2              # FPSにかけるスケールファクター
color_flag = True               # カラー動画はTrue, グレースケール動画はFalse
 
# 動画の再生速度を変更する関数を実行
m_speed_change(path_in, path_out, scale_factor, color_flag)

# 固定fpsに変換

# 全画像保存

In [10]:
import cv2
import os

def save_all_frames(video_path, dir_path, basename, ext='jpg'):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    print(dir_path)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    n = 0

    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
            n += 1
        else:
            return


save_all_frames('Mountain - 34608.mp4', '\\result', 'sample_video_img')

#save_all_frames('Mountain - 34608.mp4', '\\result_png', 'sample_video_img', 'png')

\result
